<h1><span style="color:red">Explore Your Dataset with Holoviz</span></h1>

This sample notebook will walk you through several visualizations implemented in Holoviz, http://holoviz.org/ 

Author: Enrique Sanchez

## 1. Retrieve survey parameters from the URL

In [1]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

In [2]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

# pn.extension('tabulator')
pn.extension()

def printmd(string):
    display(Markdown(string))

absolutePath = "/home/jovyan/temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint


# specific imports
import requests
import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 2. Import additional Holoviz modules

In [3]:
#!pip install panel
#!pip install holoviews
#!pip install bokeh
#!pip install hvplot
#!pip install datashader
#!pip install xarray
    
import holoviews as hv
from holoviews import opts
import hvplot.pandas
import xarray as xr
#from datashader.utils import lnglat_to_meters
import geopandas as gpd
from shapely.geometry import Point
from holoviews.element.tiles import OSM

# Loading extensions
hv.extension('bokeh')
pn.extension("tabulator")

## 3. Select a survey file from SuAVE, or a local CSV file to import

In [4]:
data_select = pn.widgets.RadioBoxGroup(name='Select notebook', options=['Load survey file from SuAVE', 
                                                                        'Import a local CSV file'], 
                                       inline=False)
data_select

RadioBoxGroup(name='Select notebook', options=['Load survey file from Su...], value='Load survey f...)

In [5]:
data_input = pn.widgets.FileInput()
    
def check_selection():
    if data_select.value == 'Load survey file from SuAVE':
        global fname
        fname = absolutePath + csv_file
        printmd("<b><span style='color:red; font-size: 150%;'>Current SuAVE survey will be loaded. Continue to step 4.</span></b>")

    else:
        message = pn.pane.HTML("<b><span style='color:red; font-size: 200%;'>Upload data and continue to step 4.</span><br><span style='font-size: 150%;'>IMPORTANT: The local CSV file should not have SuAVE-specific variable names!</span></b>", width=700)
        return pn.Column(message, data_input)
    
check_selection()

<b><span style='color:red; font-size: 150%;'>Current SuAVE survey will be loaded. Continue to step 4.</span></b>

## 4. Load the file and explore it

In [6]:
# read the csv file
original_df = panellibs.extract_data(absolutePath + csv_file)



In [7]:
if not pd.isnull(data_input.filename):
    fname = absolutePath + data_input.filename
    data_input.save(fname)

df = panellibs.extract_data(fname)

# panellibs.slider(df)

# view the dataframe
with pd.option_context("display.max_columns", None):
    if any("geometry" in col for col in df.columns):
        display(df.drop(['geometry'],axis=1))
    else:
        display(df)
    
 


,Name,OAID#link#multi,Affiliation#sortquan,City#sortquan,Region#sortquan,Country#sortquan,Latitude#hidden,Longitude#hidden,Collaborators#multi#link#sortquan,Scope#multi#sortquan,Keywords#multi#sortquan,OA concepts#multi#sortquan,Publications#hidden,Publication Dates#multi#sortquan,#img,#netvis
0,A Olioso,https://openalex.org/A4227955457,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A4227955454|https://openalex.org/A4227955461|https://openalex.org/A4227955455|https://openalex.org/A4227955463|https://openalex.org/A4227955453|https://openalex.org/A4227955464|https://openalex.org/A4227955456|https://openalex.org/A4227955462|https://openalex.org/A4227955460|https://openalex.org/A4227955459|https://openalex.org/A4227955452|https://openalex.org/A4227955458,aquifer|transboundary,NaN,Groundwater|Geology|Geotechnical engineering|Hydrology (agriculture)|Aquifer|Environmental science|Computer science|Water resource management,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A4227955457"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2021,US,02ac504b6e11517e2110d174ea70a1a7ac1cf19899e1a0f23c29558f6225db03
1,A Olioso,https://openalex.org/A4226682424,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A4226682420|https://openalex.org/A4226682425|https://openalex.org/A4226682421|https://openalex.org/A4226682429|https://openalex.org/A4226682431|https://openalex.org/A4226682426|https://openalex.org/A4226682422|https://openalex.org/A4226682428|https://openalex.org/A4226682427|https://openalex.org/A4226682419|https://openalex.org/A4226682423|https://openalex.org/A4226682430,aquifer|transboundary,NaN,Groundwater|Geology|Geotechnical engineering|Hydrology (agriculture)|Aquifer|Environmental science|Water resource management,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A4226682424"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2021,US,8f95a1d08aacc416f1abe22426fe9c9fd2f8f338bb7365407f284e3985165d23
2,A. Alassane,https://openalex.org/A2484425674,Cheikh Anta Diop University,Dakar,NaN,Senegal,14.686944,-17.463333,https://openalex.org/A2434763705|https://openalex.org/A3069707669|https://openalex.org/A2182351332|https://openalex.org/A3051995119,aquifer|transboundary,NaN,Sociology|Population|Water supply|Demography|Groundwater|Water quality|Ecology|Geology|Geotechnical engineering|Groundwater recharge|Hydrology (agriculture)|Environmental engineering|Aquifer|Biology|Environmental science|Water resource management,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A2484425674"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2010,US,f55f8f5c25002f0f2a2e121be602248623f07494d5161a13d399ab12aa746bac
3,A. Aureli,https://openalex.org/A2422334401,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A2304341794|https://openalex.org/A2182540860,aquifer|transboundary,NaN,Karst|Biology|Tourism|Business|Environmental planning|Archaeology|Environmental science|Groundwater|Geotechnical engineering|Water resources|Water resource management|Environmental resource management|Environmental protection|Law|Ecology|Engineering|Multidisciplinary approach|Aquifer|Geography|Political science,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A2422334401"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2010,US,fee72a7c6e6595abd9a1fe8878cb3c9be76652d50b3986c6b1cb4ac610869e76
4,A. Aureli,https://openalex.org/A3086349667,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A3085518772|https://openalex.org/A3085940897|https://openalex.org/A3086175637|https://openalex.org/A3086707070|https://openalex.org/A3216340081|https://openalex.org/A3084770820|https://openalex.org/A3085504345,aquifer|transboundary,NaN,Environmental resource management|Groundwater|Environmental planning|Geology|Geotechnical engineering|Hydrology (agriculture)|Aquifer|Environmental

## 5. Select variables of different types to explore

In [8]:
# Finds exisitng qualifiers/non-qualifiers in data set
qualifiers, non_qualifiers = ['---', 'None'], []
for column in original_df.columns:
    if '#' in column:
        qualifier = '#' + column.split('#', )[1]
        if qualifier in qualifiers:
            continue
        qualifiers.append(qualifier)  
    else:   
        non_qualifiers.append(column)

        
# Initializes selector widgets
# Includes: Qualifier selector, Variable selector
q_selector = pn.widgets.Select(name='1. Select Qualifier', options=qualifiers)
v_selector = pn.widgets.Select(name='2. Select Variable(s)', options=[])


# Creates link between selectors
#
# Variable selector is dependent on qualifier selector
def var_trigger(target, event):
    if event.new == 'None':
        target.options = original_df.loc[:, non_qualifiers].columns.tolist()
    else:
        is_qualifier = original_df.columns.str.contains(event.new)
        target.options = original_df.loc[:, is_qualifier].columns.tolist()
    return target

q_selector.link(v_selector, callbacks={'value': var_trigger})


# Displays dataframe and slider widget
kept = []
@pn.depends(v_selector.param.value)
def update_df(choice):
    if pd.isnull(choice):
        return;
    
    def display_df(row=0):
        temp = original_df.loc[:, kept+[choice]]
        temp = pn.widgets.Tabulator(temp, widths=150, pagination='remote', page_size=min(len(original_df.loc[row:row+4, kept+[choice]])-1,10))
        return temp
    
    return display_df()


# Keep button widget
def keep_trigger(event):
    if v_selector.value in kept:
        return
    kept.append(v_selector.value)
        
keep = pn.widgets.Button(name='Keep Variable')
keep.param.watch(keep_trigger, ['clicks'])


# Save button widget
def save_trigger(event):
    global user_df
    user_df = original_df[kept].copy()

save = pn.widgets.Button(name='Save Table')
save.param.watch(save_trigger, ['clicks'])

# Displays widgets produced above
selectors1 = pn.Column(q_selector, v_selector)
buttons = pn.Column(keep, save, margin=(10,0,0,0))
pn.Row(pn.Column(selectors1, buttons), pn.Column(update_df, margin=(0,0,0,20)))

Row
    [0] Column
        [0] Column
            [0] Select(name='1. Select Qualifier', options=['---', 'None', ...], value='---')
            [1] Select(name='2. Select Variable(s)')
        [1] Column(margin=(10, 0, 0, 0))
            [0] Button(name='Keep Variable')
            [1] Button(name='Save Table')
    [1] Column(margin=(0, 0, 0, 20))
        [0] ParamFunction(function, _pane=Str, defer_load=False)

## 6. Select dataset to analyze (original or the one just saved)

In [9]:
# Determines the existance of a user created data frame
possible_opts = ['---','Original', 'Saved']
saved = 'user_df' in globals()
original = 'original_df' in globals()
true_opts = possible_opts[:original+saved+1]

# Uses user input to choose data frame 
def df_selected(event):
    global df
    if event.new == '---':
        return
    elif event.new == 'Saved':
        df=user_df.copy()
    elif event.new == 'Original':
        df=original_df.copy()

df_selection = pn.widgets.Select(name='Select table to analyze:', options=true_opts, value="Original")
df_selection.param.watch(df_selected, ['value'])

df_selection

Select(options=['---', 'Original'], value='Original')

## 7. Statistical summaries for variables and subsets

In [10]:
# Finding quantitative and qualitative variables
#
# Determines types of values to compare variables (this cell)
#
# Determines options available for selectors in visualizations (next cell)
# e.g. quantitative for x & y (scatter), qualitative 
# for identifier
quantitative, qualitative = [], []
for column in df.columns:
    if df[column].dtype == 'O':
        qualitative.append(column)
        continue
    quantitative.append(column)

# Initializes selector widgets
# Includes: Column selector, Comparison selector, Value selector, 
#           Value slider, Info selector
comp_operators = ['None', 'less than', 'greater than', 'equal to', 'not equal to']
comparison_selector = pn.widgets.Select(name='Comparison Operator', options=comp_operators)
column_selector = pn.widgets.Select(name='Variable', options=['Entire table']+df.columns.tolist())
value_selector = pn.widgets.Select(name='Value')
value_slider = pn.widgets.FloatSlider(name='Value')
info_select = pn.widgets.Select(name='Variable Statistics', options=df.columns.tolist())


# Displays unique values for variable selected for the
# Column selector through a widget. This is either
# a slider (quantitative) or a selector (qualitative)
@pn.depends(column_selector)
def show_values(column):
    if column == 'Entire table':
        return pn.Row(value_selector, width=150)
    
    elif column in quantitative:
        value_slider.start = df[column].min()
        value_slider.end = df[column].max()
        comparison_selector.options = comp_operators
        return pn.Row(value_slider, width=150)
    
    else:
        options = df[column].unique().tolist()
        value_selector.options = ['None']+options
        comparison_selector.options = ['None', 'equal to', 'not equal to']
        return pn.Row(value_selector, width=150)

    
# Helper function – Filters data based on the
# user's selected options.
def filter_data(col, comp, val_select, val_slide):
    if col == 'Entire table':
        return df
    
    elif col in quantitative:
        if comp == 'less than':
            filtered = df[df[col] < val_slide]
        elif comp == 'greater than':
            filtered = df[df[col] > val_slide]
        elif comp == 'equal to':
            filtered = df[df[col] == val_slide]
        else:
            filtered = df[df[col] != val_slide]
        
    else:
        if comp == 'not equal to':
            filtered = df[df[col] != val_select]
        else:
            filtered = df[df[col] == val_select]
            
    return filtered

# Main function – Displays filtered data frame and column statistics.
@pn.depends(column_selector, comparison_selector, 
            value_selector, value_slider, info_select)
def display_data(col, comp, val_select, val_slide, info_col):
    
    # User selects 'Entire table'
    if col == 'Entire table':
        comparison_selector.disabled = True
        value_selector.disabled = True
        
            # Column Selector widget
        col_selection = pn.widgets.MultiSelect(options=list(df.columns.values),width=350, margin=(20,0,5,30), name='Columns', value=list(df.columns.values))
        @pn.depends(col_selection)
        def select_row2(col):
            temp = df[col]
            temp = pn.widgets.Tabulator(temp, widths=150, pagination='remote', page_size=10)
            return temp
        up_row = pn.Row(col_selection)
        
        # Column info
        info = df[[info_col]].describe().T.reset_index(drop=True)
        info_widget = pn.Row(info, margin=(-10,0,0,480))
        
        return pn.Column(info_widget, up_row, pn.panel(select_row2,width=800),width=800)
    

    # Ensures comparison and value are enabled
    elif comparison_selector.disabled:
        comparison_selector.disabled = False
        value_selector.disabled = False
    
    # Data frame not displayed if expression not complete
    if (comp == 'None') or (val_select == 'None'):
        return
    
    # Filters dataframe and creates information table
    filtered = filter_data(col, comp, val_select, val_slide)
    info = filtered[[info_col]].describe().T.reset_index(drop=True)
    info_widget = pn.Row(info, margin=(-10,0,0,480))
    
    # Row slider will not function correctly if data frame is of size 1.
    if len(filtered) <= 1:
        temp = pn.widgets.Tabulator(filtered, widths=150, pagination='remote', page_size=10)
        return pn.Column(info_widget, temp)
    else:
        
        col_sel3 = pn.widgets.MultiSelect(options=list(filtered.columns.values),width=350, margin=(20,0,5,30), name='Columns', value=list(filtered.columns.values))
        @pn.depends(col_sel3)
        def select_row3(col):
            temp = filtered[col]
            temp = pn.widgets.Tabulator(temp, widths=150, pagination='remote', page_size=10)
            return temp
    
        up_row = pn.Row(col_sel3)
        return pn.Column(info_widget, up_row, pn.panel(select_row3,width=800),width=800)

# Displays widgets produced above
var_comp = pn.Row(column_selector, comparison_selector, width=300, height=80)
expression = pn.Row(var_comp, show_values, css_classes=['widget-box'])
info = pn.Column(info_select, margin=(0,0,0,20))
pn.Column(pn.Row(expression, info), pn.panel(display_data, width=800),width=800).servable()



Column(width=800)
    [0] Row
        [0] Row(css_classes=['widget-box'])
            [0] Row(height=80, sizing_mode='fixed', width=300)
                [0] Select(name='Variable', options=['Entire table', ...], value='Entire table')
                [1] Select(disabled=True, name='Comparison Operator', options=['None', 'less than', ...], value='None')
            [1] ParamFunction(function, _pane=Row, defer_load=False)
        [1] Column(margin=(0, 0, 0, 20))
            [0] Select(name='Variable Statistics', options=['Name', 'OAID#link#multi'...], value='Name')
    [1] ParamFunction(function, _pane=Column, defer_load=False, width=800)

## 8. Visualize the dataset

In [12]:
# Finding qualitative variables that contain non-unique values
#
# Prevents groupby selector from displaying all values
# in a column containing only unique values
unique = []
for col in qualitative:
    col_data = df[col].dropna()
    size = len(col_data)
    n_unique = col_data.nunique()
    if n_unique >= 100:
        continue
    if col_data.nunique() < size-1:
        unique.append(col)


# Detects latitude and longitude columns and
# creates data frame containing these coordinates.
has_coords = False
lat, lon = [],[] 

for col in df.columns:
    if '#number#hidden' in col:
        if 'lon' in col:
            lon.append(col)
        elif 'lat' in col:
            lat.append(col)
    elif 'latitude' in col.lower():
        lat.append(col)
    elif 'longitude' in col.lower():
        lon.append(col)

if len(lat) > 0 and len(lon) > 0:
    has_coords = True
#     x, y = datashader.geo.lnglat_to_meters(df[lon[0]], df[lat[0]])
    #x, y = lnglat_to_meters(df[lon[0]], df[lat[0]])
    
    ###
    geometry = [Point(lon, lat) for lon, lat in zip(df[lon[0]], df[lat[0]])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    gdf = gdf.to_crs("EPSG:3857")
    x = gdf.geometry.x
    y = gdf.geometry.y
    ###
    coordinates = df.join([pd.DataFrame({'easting': x}), 
                           pd.DataFrame({'northing': y})])

        
# Defining available plot types – for user
uni = ['histogram', 'boxplot']
multi = ['scatter']
group = ['density']
maps = []
if has_coords:
    maps = ['map']


# Initializes selector widgets
# Includes: Plot type, X variable, Y variable, 
#           Identifer, Size, Groupby, Subgroup
p_selector = pn.widgets.Select(name='1. Plot Type', options=multi+uni+group+maps)
x_selector = pn.widgets.Select(name='2. X Variable', options=quantitative)
y_selector = pn.widgets.Select(name='3. Y Variable', options=quantitative)
identifier = pn.widgets.Select(name='Identifier', options=qualitative)
size = pn.widgets.FloatSlider(name='Size', start=3, value=6, end=12)
g_selector = pn.widgets.Select(name='Groupby (Density Only)', options=['None']+unique)
sg_selector = pn.widgets.Select(name='Subgroup', options=['None'])
dummy = pn.widgets.Select(name='Dummy')


# Helper function – Disables/enables widgets
def disabler(x, y, s, i, g, sg):
    x_selector.disabled = x
    y_selector.disabled = y
    size.disabled = s
    identifier.disabled = i
    g_selector.disabled = g
    sg_selector.disabled = sg
    
    
# Master plotting function
@pn.depends(p_selector.param.value, x_selector.param.value, y_selector.param.value,
            identifier.param.value, size.param.value, g_selector.param.value, 
            sg_selector.param.value)
def plotter(p_selector, x, y, identifier, size, group_col, sg_value):
    
    # Defining available plot types – for hvplot()
    multi = ['scatter']
    uni = ['histogram', 'boxplot']
    
    if (x == y) and not (p_selector in uni+group+maps):
        disabler(False, False, False, False, True, True)
        return
    
    # Multivariate plots
    elif p_selector in multi:
        disabler(False, False, False, False, True, True)
        ident = []
        if len(qualitative) > 0:
            ident = [identifier]
        
        # Scatter plots with more than 4000 points significantly increase lag in plot
        # interactivity. HoloViz's datashade made to alleviate these situations.
        if len(df) > 4000:
            plot = df.hvplot(x, y, hover_cols=ident, datashade=True,
                         hover_color='red', kind=p_selector).opts(frame_height=300)
        else:
            plot = df.hvplot(x, y, hover_cols=ident, hover_color='red', 
                             kind=p_selector).opts(frame_height=300, size=size)
    
    # Univariate plots
    elif p_selector in uni:
        disabler(False, True, True, True, True, True)
        choice = [i for i in ['hist', 'box'] if p_selector.startswith(i)][0]
        horizontal = False
        
        if choice == 'box':
            horizontal = True
            
        plot = df.hvplot(y=x, hover_color='red', kind=choice, invert=horizontal).opts(frame_height=300)
    
    # Density/groupby plot
    elif p_selector in group:
        disabler(False, True, True, True, False, False)
        
        if (group_col != 'None') & (sg_value != 'None'):   
            filtered_df = df[df[group_col] == sg_value]
            plot = filtered_df.hvplot(y=x, kind='kde').opts(frame_height=300)
        else:
            plot = df.hvplot(y=x, kind='kde').opts(frame_height=300)

    # Map plot
    elif p_selector in maps:
        disabler(True, True, False, False, True, True)
        ident = []
        if len(qualitative) > 0:
            ident = [identifier]
        plot = OSM() * coordinates.hvplot.points(x='easting', y='northing',
                                                 hover_cols=ident, size=size)
        
    return plot


# Produces options for 'Subgroup' after selecting groupby
def group_trigger(event):
    if g_selector.value == 'None':
        sg_selector.value == 'None'
        sg_selector.disabled = True
        return
    sg_selector.disabled = False
    sg_selector.options = ['None'] + df[g_selector.value].dropna().unique().tolist()
    
g_selector.param.watch(group_trigger, ['value'])


# Displays widgets produced above
selectors2 = pn.Row(p_selector, x_selector, y_selector)
scatter_options = pn.Column(identifier, size, margin=(40,0,0,0), 
                            width=250, css_classes=['widget-box'])

group_options = pn.Column(g_selector, sg_selector, margin=(20,0,0,0), 
                          width=250, css_classes=['widget-box'])

toolbar = pn.Column(scatter_options, group_options, margin=(0,10,0,0))

pn.Column(selectors2, pn.Row(toolbar, plotter))

Column
    [0] Row
        [0] Select(name='1. Plot Type', options=['scatter', 'histogram', ...], value='scatter')
        [1] Select(name='2. X Variable', options=['Latitude#hidden', ...], value='Latitude#hidden')
        [2] Select(name='3. Y Variable', options=['Latitude#hidden', ...], value='Latitude#hidden')
    [1] Row
        [0] Column(margin=(0, 10, 0, 0))
            [0] Column(css_classes=['widget-box'], margin=(40, 0, 0, 0), width=250)
                [0] Select(name='Identifier', options=['Name', 'OAID#link#multi'...], value='Name')
                [1] FloatSlider(end=12, name='Size', start=3, value=6)
            [1] Column(css_classes=['widget-box'], margin=(20, 0, 0, 0), width=250)
                [0] Select(disabled=True, name='Groupby (Density Only)', options=['None', 'Region#sortquan'...], value='None')
                [1] Select(disabled=True, name='Subgroup', options=['None'], value='None')
        [1] ParamFunction(function, _pane=Str, defer_load=False)